## 9. Testing and Debugging

### 81 Use `tracemalloc` to Understand Memory Usage and Leaks

```python
# waste_memory.py

import os

class MyObject:
    def __init__(self):
        self.data = os.urandom(100)

def get_data():
    values = []
    for _ in range(100):
        obj = MyObject()
        values.append(obj)
    return values

def run():
    deep_values = []
    for _ in range(100):
        deep_values.append(get_data())
    return deep_values
```

```python
# using_gc.py

import gc

found_objects = gc.get_objects()
print('Before:', len(found_objects))

import waste_memory

hold_reference = waste_memory.run()

found_objects = gc.get_objects()
print('After: ', len(found_objects))
for obj in found_objects[:3]:
    print(repr(obj)[:100])

print('...')
```

```shell
$ python using_gc.py 
Before: 4656
After:  15162
<waste_memory.MyObject object at 0x7f79d2a0d6a0>
<waste_memory.MyObject object at 0x7f79d2a0d700>
<waste_memory.MyObject object at 0x7f79d2a0d760>
...
```

```python
# top_n.py

import tracemalloc

tracemalloc.start(10)                      # Set stack depth
time1 = tracemalloc.take_snapshot()        # Before snapshot

import waste_memory

x = waste_memory.run()                     # Usage to debug
time2 = tracemalloc.take_snapshot()        # After snapshot

stats = time2.compare_to(time1, 'lineno')  # Compare snapshots
for stat in stats[:3]:
    print(stat)
```

```shell
$ python top_n.py 
/home/jay/_doc/spike/python/effective/81/waste_memory.py:22: size=2314 KiB (+2314 KiB), count=29994 (+29994), average=79 B
/home/jay/_doc/spike/python/effective/81/waste_memory.py:27: size=469 KiB (+469 KiB), count=10001 (+10001), average=48 B
/home/jay/_doc/spike/python/effective/81/waste_memory.py:28: size=82.8 KiB (+82.8 KiB), count=100 (+100), average=848 B
```

```python
# with_trace.py

import tracemalloc

tracemalloc.start(10)
time1 = tracemalloc.take_snapshot()

import waste_memory

x = waste_memory.run()
time2 = tracemalloc.take_snapshot()

stats = time2.compare_to(time1, 'traceback')
top = stats[0]
print('Biggest offender is:')
print('\n'.join(top.traceback.format()))
```

```shell
$ python with_trace.py 
Biggest offender is:
  File "with_trace.py", line 24
    x = waste_memory.run()
  File "/home/jay/_doc/spike/python/effective/81/waste_memory.py", line 34
    deep_values.append(get_data())
  File "/home/jay/_doc/spike/python/effective/81/waste_memory.py", line 27
    obj = MyObject()
  File "/home/jay/_doc/spike/python/effective/81/waste_memory.py", line 22
    self.data = os.urandom(100)
```

> - 파이썬 프로그램이 메모리를 사용하고 누수하는 양상을 이해하기는 어렵다.
> - `gc` 모듈은 어떤 객체가 존재하는지 이해할 때는 도움이 되지만, 객체가 어떻게 할당됐는지 파악할 수 있는 정보는 제공하지 않는다.
> - `tracemalloc` 내장 모듈은 프로그램이 메모리를 사용하는 이유를 알고 싶을 때 쓸 수 있는 강력한 도구다.